In [20]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

In [21]:
df = pd.read_csv("datasets/train.csv")
df.head()

,index,text,label
0,10000,निर्वाचन आयोगले गति छाड्यो,0
1,10001,जीतपुरसिमराः एमालेबाट चौधरी र अधिकारीले गरे उम...,0
2,10002,दलहरुको घरदैलो कार्यक्रममा ठुलो ठुलो आवाजमा स्...,0
3,10003,"म पनि त्यही सोचेको, अब पनि भुई कटहर लाई भोट न...",0
4,10006,शिक्षकबाट राजीनामा नदिई उपमेयरमा उम्मेदवारी दर्ता,0


In [22]:
df.dropna(inplace = True)

In [23]:
X_train, X_test, y_train, y_test = train_test_split(df["text"], df["label"], test_size=0.2, random_state=42)

In [24]:
class DevanagariDataset(Dataset):
    def __init__(self, texts, labels, max_length=100):
        self.texts = texts.tolist()  
        self.labels = labels.tolist()  
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text_tensor = torch.tensor([ord(char) for char in self.texts[idx]], dtype=torch.long)
        
        if len(text_tensor) > self.max_length:
            text_tensor = text_tensor[:self.max_length]  
        else:
            padding = torch.zeros(self.max_length - len(text_tensor), dtype=torch.long)  
            text_tensor = torch.cat((text_tensor, padding)) 

        label_tensor = torch.tensor(self.labels[idx], dtype=torch.long)

        return text_tensor, label_tensor


In [25]:
train_dataset = DevanagariDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = DevanagariDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [26]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [27]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)  
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        out, _ = self.lstm(x, (h0, c0))  
        out = out[:, -1, :]  # Take the output from the last time step
        out = self.fc(out)
        return out

In [28]:
input_size = 1  # Each character is represented by a single integer
hidden_size = 128  
num_layers = 2  
num_classes = len(df['label'].unique())  
num_epochs = 10
learning_rate = 0.001

In [29]:
model = LSTMModel(input_size, hidden_size, num_layers, num_classes)
model.to(device)

LSTMModel(
  (lstm): LSTM(1, 128, num_layers=2, batch_first=True)
  (fc): Linear(in_features=128, out_features=5, bias=True)
)

In [30]:
def train_model(model, train_loader, test_loader, criterion, optimizer, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0
        correct_train_predictions = 0
        
        for texts, labels in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs} - Training", leave=False):
            texts, labels = texts.to(device), labels.to(device)
            texts = texts.unsqueeze(-1).float() 
            outputs = model(texts)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            total_train_loss += loss.item()
            
            loss.backward()
            optimizer.step()

            _, predicted = torch.max(outputs.data, 1)
            correct_train_predictions += (predicted == labels).sum().item()
        
        train_accuracy = correct_train_predictions / len(train_loader.dataset)
        train_loss = total_train_loss / len(train_loader)
        
        model.eval()
        total_test_loss = 0
        correct_test_predictions = 0
        
        with torch.no_grad():
            for texts, labels in tqdm(test_loader, desc=f"Epoch {epoch + 1}/{num_epochs} - Testing", leave=False):
                texts, labels = texts.to(device), labels.to(device)
                texts = texts.unsqueeze(-1).float()  

                outputs = model(texts)
                loss = criterion(outputs, labels)
                total_test_loss += loss.item()
                
                _, predicted = torch.max(outputs.data, 1)
                correct_test_predictions += (predicted == labels).sum().item()
        
        test_accuracy = correct_test_predictions / len(test_loader.dataset)
        test_loss = total_test_loss / len(test_loader)
        
        print(f"Epoch {epoch + 1}/{num_epochs}")
        print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")
        print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
train_model(model, train_loader, test_loader, criterion, optimizer, num_epochs)
torch.save(model.state_dict(), "lstm_model.pth")